In [1]:
#import nécessaire
import matplotlib.pyplot as plt
from gtts import gTTS
from pygame import mixer
import os
import numpy as np
from tempfile import TemporaryFile
import speech_recognition as sr
from googletrans import Translator
from ibm_watson import NaturalLanguageUnderstandingV1
from watson_developer_cloud.natural_language_understanding_v1 import Features, SentimentOptions, EmotionOptions



#définition des variables globales
listquestions=list()
nombrequestions=list()
posneg=list()
joy=list()
anger=list()
disgust=list()
sadness=list()
fear=list()


#récupération des questions
def questions(namefolder):
    fichier=open(namefolder,'r',encoding = 'unicode_escape') 
    text=fichier.read()
    numquest=1
    for interro in text.split('?'):
        interro = interro.replace(u'\xa0', u' ')
        interro = interro.replace(u'\n', u' ')
        listquestions.append(interro)
        nombrequestions.append(numquest)
        numquest=numquest+1
    fichier.close()
    nombrequestions.remove(len(nombrequestions))
    del listquestions[len(nombrequestions)]
    return listquestions,nombrequestions


#calcul de la moyenne et création de graphique
def moyennelistes(posneg,joy,anger,disgust,sadness,fear):
    #création des variables moyennes
    resposneg=''
    mposneg=0
    mjoy=0
    manger=0
    mdisgust=0
    msadness=0
    mfear=0
    #calcul des moyennes
    for element in posneg:
        mposneg=mposneg+element
    mposneg=mposneg/len(posneg)
    if mposneg<0:
        resposneg='Aperçu de la vie négatif'
    elif mposneg>0:
        resposneg='Aperçu de la vie positif'
    else:
        resposneg='Neutre'
    for element in joy:
        mjoy=mjoy+element
    mjoy=mjoy/len(joy)
    for element in anger:
        manger=manger+element
    manger=manger/len(anger)
    for element in disgust:
        mdisgust=mdisgust+element
    mdisgust=mdisgust/len(disgust)
    for element in sadness:
        msadness=msadness+element
    msadness=msadness/len(sadness)
    for element in fear:
        mfear=mfear+element
    mfear=mfear/len(fear)
    #création des graphiques
    #graphique en fonction des résultats
    plt.figure(1)
    plt.title("Résultats du test")
    vnombrequest=np.array(nombrequestions)
    vjoy=np.array(joy)
    vanger=np.array(anger)
    vdisgust=np.array(disgust)
    vsadness=np.array(sadness)
    vfear=np.array(fear)
    plt.plot(vnombrequest,vjoy,'g',label="Joie")
    plt.plot(vnombrequest,vanger,'r',label="Colère")
    plt.plot(vnombrequest,vdisgust,'c',label="Dégoût")
    plt.plot(vnombrequest,vsadness,'m',label="Tristesse")
    plt.plot(vnombrequest,vfear,'y',label="Peur")
    plt.axis([0, len(nombrequestions), 0, 1])
    plt.xlabel('Numéro des questions')
    plt.ylabel('Résultats')
    plt.legend()
    plt.show()
    #graphique en fonction des moyennes
    plt.figure(2)
    plt.title("Résultats du test émotion")
    labels = 'Joie', 'Colère', 'Dégoût', 'Tristesse', 'Peur'
    plt.axis("equal")
    plt.pie([mjoy, manger, mdisgust, msadness, mfear],labels=labels,autopct="%1.1f")  
    plt.show()
    #affichage positif ou negatif 
    print(resposneg)
    texttradaudio2 = gTTS(text=resposneg, lang='fr', slow=False)
    mixer.init()
    sf2 = TemporaryFile()
    texttradaudio2.write_to_fp(sf2)
    sf2.seek(0)
    mixer.music.load(sf2)
    mixer.music.play()


#application principale
questions("C:/Users/camil/Desktop/Projetmonsprojetbase/questionslambdas.txt")
for ask in listquestions:
    texttradaudio = gTTS(text=ask, lang='fr', slow=False)
    mixer.init()
    sf = TemporaryFile()
    texttradaudio.write_to_fp(sf)
    sf.seek(0)
    mixer.music.load(sf)
    mixer.music.play()
    print(ask)    
    #enregistrement réponse 
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Dites queslques chose")
        audiofinal = r.listen(source)
    text = r.recognize_google(audiofinal,language='fr-FR')
    translator = Translator()
    translations = translator.translate(text)
    KEY_IBM = "74v192fabJsRIZYIahiYu15mrzTZ1F0ISEA9hCGIKU5t"
    URL_IBM="https://gateway-wdc.watsonplatform.net/natural-language-understanding/api"
    texttrad = translations.text
    print(texttrad)
    alchemy_language = NaturalLanguageUnderstandingV1(version='2018-11-16',iam_apikey=KEY_IBM,url=URL_IBM)
    sentiment = alchemy_language.analyze(text=texttrad,features=Features(emotion=EmotionOptions(),sentiment=SentimentOptions())).get_result()
    print(sentiment)
    #remplissage des listes des résultats
    emo = sentiment['emotion']
    emo2 = emo['document']
    emo3 = emo2['emotion']
    joy.append(emo3['joy'])
    anger.append(emo3['anger'])
    disgust.append(emo3['disgust'])
    sadness.append(emo3['sadness'])
    fear.append(emo3['fear'])
    sent = sentiment['sentiment']
    sent2 = sent['document']
    sent3 = sent2['score']
    posneg.append(sent3)
moyennelistes(posneg,joy,anger,disgust,sadness,fear)   

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
Comment tappelles-tu 
Dites queslques chose
what's your name
{'usage': {'text_units': 1, 'text_characters': 16, 'features': 2}, 'sentiment': {'document': {'score': 0, 'label': 'neutral'}}, 'language': 'en', 'emotion': {'document': {'emotion': {'sadness': 0.0, 'joy': 0.0, 'fear': 0.0, 'disgust': 0.0, 'anger': 0.0}}}}
 Quel âge as-tu 
Dites queslques chose
I'm 21


ApiException: Error: not enough text for language id, Code: 422 , X-global-transaction-id: 6a0d0f492e9902b0cc189a61d5f7e63f